In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Set the directory path and filenames
dir_path = 'C:/1. Power grid frequency data/'
file_names = ['ES_PM01.csv', 'IS02.csv', 'IRL01.csv']

# Load the CSV files into dataframes
dfs = []
for file_name in file_names:
    file_path = dir_path + file_name
    df = pd.read_csv(file_path, sep=';', 
                     usecols=[0,1,2], names=['Time','f50','QI'],
                     header=0)
    df.iloc[:,1]=df.iloc[:,1]/1000+50
    dfs.append(df)


# Only select quality QI=0
flt_dfs = []
for df in dfs:
    flt_df = df[df['QI'] == 0]
    flt_dfs.append(flt_df)
    
region_dict = {0: 'Mallorcaa', 1: 'Iceland', 2: 'Ireland'}

# Group the dataframes by region
region_groups = [df.groupby(lambda x: region_dict[i]) for i, df in enumerate(flt_dfs)]

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def LTtest (data):
    # Compute Fourier transform 
    fft_0 = np.fft.fft(data)
    
    # Randomize phases of Fourier coefficients
    rand_phases = np.random.uniform(0, 2*np.pi, size=len(fft_0))
    surrogate = np.abs(fft_0) * np.exp(1j * rand_phases)
    
    # Compute inverse Fourier transform to obtain surrogate data
    surrogate = np.real(np.fft.ifft(surrogate))
    
    data = data[~np.isnan(data)]
    L = len(data)
    
    tau = np.arange(1, 82801,900 )
    #tau = np.arange(0, L//2, 1800)
    res_1 = np.zeros(len(tau))
    res_2 = np.zeros(len(tau))
    surr_1 = np.zeros(len(tau))
    surr_2 = np.zeros(len(tau))

    for i in range(len(tau)):
        x_t = data[0 :L-tau[i]]
        x_tau = data[tau[i]:L]
        y_t = surrogate[0:(L-tau[i])]
        y_tau = surrogate[(tau[i]):L]


        # First method LT1
        res_1[i] = np.mean(x_t**2 * x_tau)-np.mean(x_t * x_tau**2)
        surr_1[i] = np.mean(y_t**2 * y_tau)-np.mean(y_t *y_tau**2)

        # Second method LT2
        res_2[i] = np.mean((x_t-x_tau)**3)/np.mean((x_t-x_tau)**2)
        surr_2[i] = np.mean((y_t-y_tau)**3)/np.mean((y_t-y_tau)**2)
        
    # Calculate the rmse(LT1)
    mse_lt1 = mean_squared_error(np.nan_to_num(res_1), np.nan_to_num(surr_1))
    rmse_lt1 = np.sqrt(mse_lt1)
        
    # Calculate the rmse(LT2)
    mse_lt2 = mean_squared_error(np.nan_to_num(res_2), np.nan_to_num(surr_2))
    rmse_lt2 = np.sqrt(mse_lt2)
    
    return res_1,surr_1,res_2,surr_2,rmse_lt1,rmse_lt2

In [3]:
mal_res_1,mal_surr_1,mal_res_2,mal_surr_2,mal_rmse_lt1,mal_rmse_lt2 = LTtest(flt_dfs[0].f50.dropna().values)
ice_res_1,ice_surr_1,ice_res_2,ice_surr_2,ice_rmse_lt1,ice_rmse_lt2 = LTtest(flt_dfs[1].f50.dropna().values)
ire_res_1,ire_surr_1,ire_res_2,ire_surr_2,ire_rmse_lt1,ire_rmse_lt2 = LTtest(flt_dfs[2].f50.dropna().values)


In [ ]:
euro_rmse_lt1=[mal_rmse_lt1,ice_rmse_lt1,ire_rmse_lt1]
euro_rmse_lt2=[mal_rmse_lt2,ice_rmse_lt2,ire_rmse_lt2]
%store euro_rmse_lt1
%store euro_rmse_lt2